<a href="https://colab.research.google.com/github/vinay18-irpanwar/Google-Colab/blob/main/Expense(presistance)_LANGGRAPH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
!pip install langgraph

In [87]:
from typing_extensions import TypedDict
#state

class Expensestate(TypedDict):
  income:float
  expense:float
  balance:float

In [88]:
#stateGraph
from langgraph.graph import StateGraph,START,END

workflow=StateGraph(Expensestate)

In [89]:
#add nodes
def income(state:Expensestate):
  total=state.get('balance',0)
  income=state.get('income',0)
  state['balance']=income+total
  return state

def expense(state:Expensestate):

  balance=state.get('balance',0)
  expense=state.get('expense',0)
  total=(balance - expense)
  state['balance']=total
  #print(f"----------------{state['balance']}----------------")
  return state


workflow.add_node("income",income)
workflow.add_node("expense",expense)


In [90]:






workflow.add_edge(START,"income")
workflow.add_edge("income","expense")
workflow.add_edge("expense",END)


# ADD memory ---> **persistance**

In [91]:
from langgraph.checkpoint.memory import MemorySaver
memory=MemorySaver()

In [92]:
app=workflow.compile(checkpointer=memory)

# config

In [93]:
config={"configurable":{"thread_id":"user1"}}

In [94]:
print("------------------user1------------------")
input1=({"income":500,"expense":200})
res=app.invoke(input1,config=config)
print(res)
input2=({"income":600,"expense":200})
result=app.invoke(input2,config=config)
print(result)
print("------------------user2------------------")
config1={"configurable":{"thread_id":"user2"}}
r=app.invoke({"income":500,"expense":100},config=config1)
print(r)

------------------user1------------------
{'income': 500, 'expense': 200, 'balance': 300}
{'income': 600, 'expense': 200, 'balance': 700}
------------------user2------------------
{'income': 500, 'expense': 100, 'balance': 400}
